In [3]:
import pandas as pd
import os
#cerrega todo os arquivos
dfsNew = []
dfsOld = []

for arq in os.listdir("./data/IT/"):
    if arq.find("New") > 0:
        dfsNew.append(pd.read_csv("./data/IT/"+arq, sep="\t"))
    elif arq.find("Old") > 0:
        dfsOld.append(pd.read_csv("./data/IT/"+arq, sep="\t"))


In [4]:
from nltk import ngrams
from sacremoses import MosesTokenizer

def tokenize(text, lang, nGram=3):
    entok = MosesTokenizer(lang=lang)
    text = entok.tokenize(text, escape=False)
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(hypothesys, references, nGram, lang):

  overlaps = []
  for h, r in zip([hypothesys], [references]):
    if (h == "") or (r == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, lang, nGram)
    b = tokenize(r, lang, nGram)

    if len(a) >= len(b) and len(a) > 0:
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a) and len(b) > 0:
      overlaps.append(len(set(a) & set(b))/len(b))
    elif len(a) == 0 or len(b) == 0:
      overlaps.append(0)

  return overlaps[0]

import numpy as np

def getStats(dfA, dfB, lang):
    df = dfA.merge(dfB, on=["livro", "capitulo", "versiculo"])

    df["sourceLen"] = df["texto_x"].apply(lambda x: getSizeSentece(x))
    df["targetLen"] = df["texto_y"].apply(lambda x: getSizeSentece(x))
    
    df["overlap"] = df.apply(lambda x: getNgramOverlap(x["texto_x"], x["texto_y"], 3, lang), axis=1)

    return df
#Matriz do novo testamento

def getSizeSentece(text):
    try:
        return len(text.split(" "))
    except:
        return 0
    

In [5]:
print("Novo testamento: ")
import copy
concat_df_new = []
new = copy.deepcopy(dfsNew)
for dfA in new:
    new = new[1:]
    for dfB in new:
             
        print(set(dfA["livro"]) - set(dfB["livro"]))
        concat_df_new.append(getStats(dfA, dfB, "it"))

Novo testamento: 
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


5

In [7]:
len(concat_df_new)

10

In [5]:
dfConcat_novo = pd.concat(concat_df_new, ignore_index=False)
print("tamanho sem cortes: ", dfConcat_novo.shape)
dfConcat_novo["VERSAO"] = "NOVO"

tamanho sem cortes:  (75339, 10)


26.448360079109094
21.66498095276019


In [6]:
print("Velho testamento: ")
import copy
concat_df_old = []
old = copy.deepcopy(dfsOld)
for dfA in old:
    old = old[1:]
    for dfB in old:
             
        print(set(dfA["livro"]) - set(dfB["livro"]))
        concat_df_old.append(getStats(dfA, dfB, "it"))

Velho testamento: 
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [7]:
len(dfsOld)

5

In [8]:
dfConcat_old = pd.concat(concat_df_old, ignore_index=False)
print("tamanho sem cortes: ", dfConcat_old.shape)
dfConcat_old["VERSAO"] = "VELHO"

tamanho sem cortes:  (206164, 10)


In [9]:
df_geral = pd.concat([dfConcat_old,dfConcat_novo])

print(df_geral["sourceLen"].mean())
print(df_geral["targetLen"].mean())
print(df_geral["overlap"].mean())

19.691001516857725
25.359441995289572
0.3367626237444126


In [10]:
df_geral

,estilo_x,livro,capitulo,versiculo,texto_x,estilo_y,texto_y,sourceLen,targetLen,overlap,VERSAO
0,BDG,Genesi,1,1,NaN,CEI,In principio Dio creò il cielo e la terra .,0,10,0.000000,VELHO
1,BDG,Genesi,1,2,NaN,CEI,Ora la terra era informe e deserta e le tenebr...,0,22,0.000000,VELHO
2,BDG,Genesi,1,3,NaN,CEI,Dio disse : « Sia la luce ! » . E la luce fu .,0,15,0.000000,VELHO
3,BDG,Genesi,1,4,NaN,CEI,Dio vide che la luce era cosa buona e separò l...,0,14,0.000000,VELHO
4,BDG,Genesi,1,5,NaN,CEI,e chiamò la luce giorno e le tenebre notte . E...,0,20,0.000000,VELHO
...,...,...,...,...,...,...,...,...,...,...,...
7950,NR1994,Apocalisse,22,18,Io lo dichiaro a chiunque ode le parole della ...,NR2006,Io lo dichiaro a chiunque ode le parole della ...,32,32,0.952381,NOVO
7951,NR1994,Apocalisse,22,19,se qualcuno toglie qualcosa dalle parole del l...,NR2006,TR e delle cose che sono descritte … - Apocalisse,33,10,0.056338,NOVO
7952,NR1994,Apocalisse,22,20,"Colui che attesta queste cose , dice : « Sí , ...",NR2006,"TR e M Sì , vieni , Signore Gesù . - Apocalisse",22,12,0.069767,NOVO
7953,NR1994,Apocalisse,22,21,La grazia del Signore Gesú sia con tutti . Foo...,NR2006,TR sia con tutti voi ; M sia con tutti i santi .,12,13,0.240000,NOVO


In [11]:
df_geral.dropna(inplace=True)
df_geral = df_geral[(df_geral.sourceLen >= 5) & (df_geral.targetLen >= 5) ]

In [12]:


selRows = df_geral[(df_geral["livro"] == "Matteo") & (df_geral["capitulo"] == 1)  & (df_geral["versiculo"] >= 1) & (df_geral["versiculo"] <= 16)].index
df_geral.drop(selRows, axis=0,inplace=True)

selRows = df_geral[(df_geral["livro"] == "Luca") & (df_geral["capitulo"] == 3)  & (df_geral["versiculo"] >= 23) & (df_geral["versiculo"] <= 38)].index
df_geral.drop(selRows, axis=0,inplace=True)

selRows = df_geral[(df_geral["livro"] == "Genesi") & (df_geral["capitulo"] == 5)  & (df_geral["versiculo"] >= 1) & (df_geral["versiculo"] <= 32)].index
df_geral.drop(selRows, axis=0,inplace=True)

selRows = df_geral[(df_geral["livro"] == "Genesi") & (df_geral["capitulo"] == 10)  & (df_geral["versiculo"] >= 1) & (df_geral["versiculo"] <= 32)].index
df_geral.drop(selRows, axis=0,inplace=True)


filter = df_geral['livro'].str.contains("2%20Cronache")
df_geral = df_geral[~filter]
filter = df_geral['livro'].str.contains("1%20Cronache")
df_geral = df_geral[~filter]


filter = df_geral['texto_x'].str.contains("Copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_x'].str.contains("copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_x'].str.contains("®")
df_geral = df_geral[~filter]
filter = df_geral['texto_x'].str.contains("&#124; Romani &#124; &#124")
df_geral = df_geral[~filter]


filter = df_geral['texto_y'].str.contains("Copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_y'].str.contains("copyright")
df_geral = df_geral[~filter]
filter = df_geral['texto_y'].str.contains("®")
df_geral = df_geral[~filter]
filter = df_geral['texto_y'].str.contains("&#124; Romani &#124; &#124")
df_geral = df_geral[~filter]

print(df_geral["sourceLen"].mean())
print(df_geral["targetLen"].mean())
print(df_geral["overlap"].mean())

C:\Users\pablo\AppData\Local\Temp\ipykernel_22548\3405594034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral.drop(selRows, axis=0,inplace=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_22548\3405594034.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral.drop(selRows, axis=0,inplace=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_22548\3405594034.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral.dro

27.108857917362613
26.650935078448015
0.4779163900140535


In [13]:
df_geral

,estilo_x,livro,capitulo,versiculo,texto_x,estilo_y,texto_y,sourceLen,targetLen,overlap,VERSAO
16,CEI,Genesi,1,17,Dio le pose nel firmamento del cielo per illum...,LND,E DIO li mise nel firmamento dei cieli per far...,11,14,0.185185,VELHO
17,CEI,Genesi,1,18,e per regolare giorno e notte e per separare l...,LND,"per governare il giorno e la notte , e separar...",22,23,0.488889,VELHO
18,CEI,Genesi,1,19,E fu sera e fu mattina : quarto giorno .,LND,"Così fu sera , e fu mattina : il quarto giorno .",10,12,0.608696,VELHO
19,CEI,Genesi,1,20,Dio disse : « Le acque brulichino di esseri vi...,LND,Poi DIO disse : « Brulichino le acque di molti...,24,29,0.460317,VELHO
20,CEI,Genesi,1,21,Dio creò i grandi mostri marini e tutti gli es...,LND,Così DIO creò i grandi animali acquatici e tut...,41,44,0.367816,VELHO
...,...,...,...,...,...,...,...,...,...,...,...
7949,NR1994,Apocalisse,22,17,( C ) Lo Spirito e la * sposa dicono : « Vieni...,NR2006,( C ) Lo Spirito e la sposa dicono : « Vieni !...,42,42,0.696629,NOVO
7950,NR1994,Apocalisse,22,18,Io lo dichiaro a chiunque ode le parole della ...,NR2006,Io lo dichiaro a chiunque ode le parole della ...,32,32,0.952381,NOVO
7951,NR1994,Apocalisse,22,19,se qualcuno toglie qualcosa dalle parole del l...,NR2006,TR e delle cose che sono descritte … - Apocalisse,33,10,0.056338,NOVO
7952,NR1994,Apocalisse,22,20,"Colui che attesta queste cose , dice : « Sí , ...",NR2006,"TR e M Sì , vieni , Signore Gesù . - Apocalisse",22,12,0.069767,NOVO


In [14]:
df_geral.to_csv("./data/FILTERED/italian.tsv", sep="\t")